In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import argparse
import sys
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_svmlight_file
import sys
sys.path.append('../')

from cs_opl import op_learning
from experiment_learning import *

In [17]:
data_name = 'satimage'
num_trials = 1
sample_size = 300

if data_name == 'satimage':
    data_name = 'satimage.scale'
elif data_name == 'vehicle':
    data_name = 'vehicle.scale'

alphas = [0.7, 0.4, 0.0]

tau_list = np.zeros(num_trials)
res_ipw3_list = np.zeros((num_trials, len(alphas)))
res_dm_list = np.zeros((num_trials, len(alphas)))
res_dml2_list = np.zeros((num_trials, len(alphas)))

res_ipw3_sn_list = np.zeros((num_trials, len(alphas)))
res_dml2_sn_list = np.zeros((num_trials, len(alphas)))

for trial in range(num_trials):
    X, Y, Y_matrix, train_test_split, classes, N, N_train, N_test = data_generation(data_name, sample_size)

    X_train, X_test = X[train_test_split], X[~train_test_split]

    Y_matrix_train, Y_matrix_test = Y_matrix[train_test_split], Y_matrix[~train_test_split]

    for idx_alpha in  range(len(alphas)):    
        alpha = alphas[idx_alpha]

        pi_behavior, pi_evaluation  = behavior_and_evaluation_policy(X, Y, train_test_split, classes, alpha=alpha)

        pi_behavior_train = pi_behavior[train_test_split]
        pi_evaluation_train, pi_evaluation_test = pi_evaluation[train_test_split], pi_evaluation[~train_test_split]

        tau = true_value(Y_matrix_test, pi_evaluation_test, N_test)
        tau_list[trial] = tau

        perm = np.random.permutation(N_train)
        X_seq_train, Y_matrix_seq_train, pi_behavior_seq_train, pi_evaluation_seq_train = X_train[perm], Y_matrix_train[perm], pi_behavior_train[perm], pi_evaluation_train[perm]

        Y_historical_matrix = np.zeros(shape=(N_train, len(classes)))
        A_historical_matrix = np.zeros(shape=(N_train, len(classes)))

        for i in range(N_train):
            a = np.random.choice(classes, p=pi_behavior_seq_train[i])
            Y_historical_matrix[i, a] = Y_matrix_seq_train[i, a]
            A_historical_matrix[i, a] = 1
            
        estimators = op_learning(X_seq_train, A_historical_matrix, Y_historical_matrix, X_test, classes)
        
        estimators.ipw_est_parameters()
        epol_ipw = estimators.ipw_fit(folds=5, algorithm='Ridge', self_norm=False)
        #epol_ipw_sn = estimators.ipw_fit(folds=5, algorithm='Ridge', self_norm=True)

        estimators.dm_est_parameters()
        epol_dm = estimators.dm_fit(folds=5, algorithm='Ridge')
        
        estimators.dml_est_parameters(folds=5, algorithm='Ridge')
        epol_dml = estimators.dml_fit(folds=5, algorithm='Ridge', self_norm=False)
        epol_dml_sn = estimators.dml_fit(folds=5, algorithm='Ridge', self_norm=True)

        res_ipw3 =  true_value(Y_matrix_test, epol_ipw, N_test)
        res_ipw3_sn =  true_value(Y_matrix_test, epol_ipw_sn, N_test)
        res_dm =  true_value(Y_matrix_test, epol_dm, N_test)
        res_dml2 =  true_value(Y_matrix_test, epol_dml, N_test)
        res_dml2_sn =  true_value(Y_matrix_test, epol_dml_sn, N_test)

        print('trial', trial)
        print('True:', tau)
        print('IPW3:', res_ipw3)
        print('IPW3_SN:', res_ipw3_sn)
        print('DM:', res_dm)
        print('DML2:', res_dml2)
        print('DML2_SN:', res_dml2_sn)

        res_ipw3_list[trial, idx_alpha] = res_ipw3
        res_ipw3_sn_list[trial, idx_alpha] = res_ipw3_sn
        res_dm_list[trial, idx_alpha] = res_dm
        res_dml2_list[trial, idx_alpha] = res_dml2
        res_dml2_sn_list[trial, idx_alpha] = res_dml2_sn

        np.savetxt("exp_results/true_value_%s.csv"%data_name, tau_list, delimiter=",")
        np.savetxt("exp_results/res_opl_ipw3_%s.csv"%data_name, res_ipw3_list, delimiter=",")
        np.savetxt("exp_results/res_opl_ipw3_sn_%s.csv"%data_name, res_ipw3_sn_list, delimiter=",")
        np.savetxt("exp_results/res_opl_dm_%s.csv"%data_name, res_dm_list, delimiter=",")
        np.savetxt("exp_results/res_opl_dml2_%s.csv"%data_name, res_dml2_list, delimiter=",")
        np.savetxt("exp_results/res_opl_dml2_sn_%s.csv"%data_name, res_dml２_sn_list, delimiter=",")

/Users/mkato/opt/anaconda3/lib/python3.7/site-packages/statsmodels/nonparametric/kernel_regression.py:237: RuntimeWarning: invalid value encountered in true_divide
  B_x = (G_numer * d_fx - G_denom * d_mx) / (G_denom**2)
/Users/mkato/opt/anaconda3/lib/python3.7/site-packages/statsmodels/nonparametric/kernel_regression.py:224: RuntimeWarning: invalid value encountered in true_divide
  G = G_numer / G_denom
/Users/mkato/opt/anaconda3/lib/python3.7/site-packages/statsmodels/nonparametric/kernel_regression.py:236: RuntimeWarning: invalid value encountered in true_divide
  B_x = d_mx / f_x - G * d_fx / f_x


(168, 6)
(211, 6)
score0 27359.303796317326
score 1056.1160374137462
(168, 6)
(211, 6)
score0 27838.037061168714
score 1033.8055298796578
(168, 6)
(211, 6)
score0 26664.076307578336
score 626.3835243022693
(168, 6)
(211, 6)
score0 23998.46423957781
score 65.8485918617302
(168, 6)
(211, 6)
score0 23997.608269654982
score 65.88089448403151
(168, 6)
(211, 6)
score0 29906.53061959688
score 31.408788871614746
(168, 6)
(211, 6)


KeyboardInterrupt: 

In [19]:
np.max(estimators.r_ker_matrix)

9.386393573091611

In [27]:
estimators.ipw_fit(folds=5, algorithm='Ridge', self_norm=False)

(168, 6)
(211, 6)
score0 0.9312234328846003
score 0.04725927146712045
(168, 6)
(211, 6)


KeyboardInterrupt: 

In [7]:
epol_dm = estimators.dm_fit(folds=5, algorithm='Ridge')

score0 0.42478432778085706
score 0.624844937256635
score0 0.4169980922742169
score 0.6125833068164179
score0 0.36226943390047933
score 0.5264911663356887
score0 0.14679332839448594
score 0.18867976091706315
score0 0.1225738767387439
score 0.1523443168148735
score0 0.46360129885678664
score 0.4700256817789767
score0 0.4557275655349157
score 0.46093319162755697
score0 0.40036343525716633
score 0.39725590502645886
score0 0.1658864454415211
score 0.13029073745882658
score0 0.13583282014090733
score 0.10056311960314633
score0 0.4329068893226889
score 0.592541037800257
score0 0.4250849108688239
score 0.5816477801009194
score0 0.3700894471683462
score 0.5051713282700121
score0 0.14666226511271596
score 0.19583761696619295
score0 0.12039701272586464
score 0.16146765568657995
score0 0.4991150808752562
score 0.3286647749210678
score0 0.4910842878571196
score 0.32437816326488206
score0 0.4346535675851164
score 0.2941034923060732
score0 0.16665710733232608
score 0.14897437249956058
score0 0.129755

In [13]:
estimators.dml_est_parameters(folds=5, method='Ridge')
epol_dml = estimators.dml_fit(folds=5, algorithm='Ridge', self_norm=False)
epol_dml_sn = estimators.dml_fit(folds=5, algorithm='Ridge', self_norm=True)

/Users/mkato/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/mkato/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/mkato/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/mkato/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/mkato/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarni

/Users/mkato/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/mkato/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/mkato/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/mkato/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/mkato/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarni

/Users/mkato/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/mkato/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/mkato/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/mkato/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/mkato/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarni

/Users/mkato/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  positive)
/Users/mkato/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  positive)
/Users/mkato/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  positive)
/Users/mkato/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  positive)
/Use

/Users/mkato/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  positive)
/Users/mkato/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  positive)
/Users/mkato/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  positive)
/Users/mkato/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  positive)
/Use

...

/Users/mkato/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  positive)
/Users/mkato/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  positive)
/Users/mkato/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  positive)
/Users/mkato/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  positive)
/Use

RuLSIF starting...
Searching for the optimal sigma and lambda...
sigma = 0.00100, lambda = 0.00100, score = 0.00000
sigma = 0.00100, lambda = 0.01000, score = 0.00000
sigma = 0.00100, lambda = 0.10000, score = 0.00000
sigma = 0.00100, lambda = 1.00000, score = 0.00000
sigma = 0.00100, lambda = 10.00000, score = 0.00000
sigma = 0.00100, lambda = 100.00000, score = 0.00000
sigma = 0.00100, lambda = 1000.00000, score = 0.00000
sigma = 0.00100, lambda = 10000.00000, score = 0.00000
sigma = 0.00100, lambda = 100000.00000, score = 0.00000
sigma = 0.00100, lambda = 1000000.00000, score = 0.00000
sigma = 0.00100, lambda = 10000000.00000, score = 0.00000
sigma = 0.00100, lambda = 100000000.00000, score = 0.00000
sigma = 0.00100, lambda = 1000000000.00000, score = 0.00000
sigma = 0.01000, lambda = 0.00100, score = -0.00000
sigma = 0.01000, lambda = 0.01000, score = -0.00000
sigma = 0.01000, lambda = 0.10000, score = -0.00000
sigma = 0.01000, lambda = 1.00000, score = 0.00000
sigma = 0.01000, lam

sigma = 100000000.00000, lambda = 0.00100, score = -0.49990
sigma = 100000000.00000, lambda = 0.01000, score = -0.49990
sigma = 100000000.00000, lambda = 0.10000, score = -0.49988
sigma = 100000000.00000, lambda = 1.00000, score = -0.49963
sigma = 100000000.00000, lambda = 10.00000, score = -0.49120
sigma = 100000000.00000, lambda = 100.00000, score = -0.32956
sigma = 100000000.00000, lambda = 1000.00000, score = -0.06485
sigma = 100000000.00000, lambda = 10000.00000, score = -0.00712
sigma = 100000000.00000, lambda = 100000.00000, score = -0.00072
sigma = 100000000.00000, lambda = 1000000.00000, score = -0.00007
sigma = 100000000.00000, lambda = 10000000.00000, score = -0.00001
sigma = 100000000.00000, lambda = 100000000.00000, score = -0.00000
sigma = 100000000.00000, lambda = 1000000000.00000, score = -0.00000
sigma = 1000000000.00000, lambda = 0.00100, score = -0.49990
sigma = 1000000000.00000, lambda = 0.01000, score = -0.49990
sigma = 1000000000.00000, lambda = 0.10000, score = -

score0 0.6062741117273958
score 0.8408967368339046
(168, 6)
(211, 6)
score0 0.410839182896669
score 0.5885229871963211
(168, 6)
(211, 6)
score0 0.17278344782393232
score 0.3680964228764057
5
4
1
(168, 210)
(168, 211)
(168, 6)
(211, 6)
score0 1.0194233378928752
score 0.7713299718038832
(168, 6)
(211, 6)
score0 0.9782303927203106
score 0.7524319875156771
(168, 6)
(211, 6)
score0 0.8115409602196435
score 0.6695104840191504
(168, 6)
(211, 6)
score0 0.47873103501818454
score 0.3921199363864484
(168, 6)
(211, 6)
score0 0.22447364873444658
score 0.17142738473650992
5
4
1
(168, 210)
(168, 211)
(168, 6)
(211, 6)
score0 1.0639959657244153
score 0.6261442727160811
(168, 6)
(211, 6)


KeyboardInterrupt: 

In [16]:
1/0.01

100.0

In [8]:
res_dm =  true_value(Y_matrix_test, epol_dm, N_test)

In [9]:
res_dm

0.1682317440185917

In [16]:
estimators = op_learning(X_seq_train, A_historical_matrix, Y_historical_matrix, X_test, classes)

In [17]:
estimators.ipw_est_parameters()

In [38]:
a, b= np.where(estimators.A ==1)

In [39]:
c = np.argmax(estimators.bpol_hat_kernel, axis=1)

In [41]:
np.mean(b == c)

0.8571428571428571

In [33]:
estimators.ipw_fit()

(168, 6)
(211, 6)
score0 2407.5609124271787
score 173.83212366984492
(168, 6)
(211, 6)
score0 2407.5618605676373
score 173.8322122110753
(168, 6)
(211, 6)
score0 2407.4187763277655
score 173.8213224689524
(168, 6)
(211, 6)
score0 2407.4829438586557
score 173.82564338621984
(168, 6)
(211, 6)


KeyboardInterrupt: 

In [30]:
np.exp(5)

148.4131591025766

In [144]:
a = estimators.ipw_fit(folds=5, algorithm='Ridge', self_norm=False)

(168, 6)
(211, 6)


../cs_opl.py:535: RuntimeWarning: overflow encountered in exp
  epol_hst = np.exp(-np.dot(x, beta))
../cs_opl.py:536: RuntimeWarning: invalid value encountered in true_divide
  epol_hst = (epol_hst.T/np.sum(epol_hst, axis=1)).T
../cs_opl.py:535: RuntimeWarning: overflow encountered in exp
  epol_hst = np.exp(-np.dot(x, beta))
../cs_opl.py:536: RuntimeWarning: invalid value encountered in true_divide
  epol_hst = (epol_hst.T/np.sum(epol_hst, axis=1)).T
../cs_opl.py:535: RuntimeWarning: overflow encountered in exp
  epol_hst = np.exp(-np.dot(x, beta))
../cs_opl.py:536: RuntimeWarning: invalid value encountered in true_divide
  epol_hst = (epol_hst.T/np.sum(epol_hst, axis=1)).T


score0 nan
score nan
(168, 6)
(211, 6)


../cs_opl.py:535: RuntimeWarning: overflow encountered in exp
  epol_hst = np.exp(-np.dot(x, beta))
../cs_opl.py:536: RuntimeWarning: invalid value encountered in true_divide
  epol_hst = (epol_hst.T/np.sum(epol_hst, axis=1)).T
../cs_opl.py:535: RuntimeWarning: overflow encountered in exp
  epol_hst = np.exp(-np.dot(x, beta))
../cs_opl.py:536: RuntimeWarning: invalid value encountered in true_divide
  epol_hst = (epol_hst.T/np.sum(epol_hst, axis=1)).T


score0 nan
score nan
(168, 6)
(211, 6)
score0 44.80854296996513
score 1.308881683828798
(168, 6)
(211, 6)
score0 43.75970334024666
score 1.281926501730938
(168, 6)
(211, 6)
score0 37.000672326075744
score 1.3933336843048583
(168, 6)
(211, 6)
score0 32.035081210935836
score 1.1549892784706297
(168, 6)
(211, 6)


...

../cs_opl.py:535: RuntimeWarning: overflow encountered in exp
  epol_hst = np.exp(-np.dot(x, beta))
../cs_opl.py:536: RuntimeWarning: invalid value encountered in true_divide
  epol_hst = (epol_hst.T/np.sum(epol_hst, axis=1)).T
../cs_opl.py:535: RuntimeWarning: overflow encountered in exp
  epol_hst = np.exp(-np.dot(x, beta))
../cs_opl.py:536: RuntimeWarning: invalid value encountered in true_divide
  epol_hst = (epol_hst.T/np.sum(epol_hst, axis=1)).T


score0 nan
score nan
(168, 6)
(211, 6)
score0 43.38201695690485
score 0.028327274226219345
(168, 6)
(211, 6)
score0 42.64913940074736
score 0.02774498698525023
(168, 6)
(211, 6)
score0 37.50440384881384
score 0.02449441068755044
(168, 6)
(211, 6)


../cs_opl.py:535: RuntimeWarning: overflow encountered in exp
  epol_hst = np.exp(-np.dot(x, beta))
../cs_opl.py:536: RuntimeWarning: invalid value encountered in true_divide
  epol_hst = (epol_hst.T/np.sum(epol_hst, axis=1)).T
../cs_opl.py:535: RuntimeWarning: overflow encountered in exp
  epol_hst = np.exp(-np.dot(x, beta))
../cs_opl.py:536: RuntimeWarning: invalid value encountered in true_divide
  epol_hst = (epol_hst.T/np.sum(epol_hst, axis=1)).T


score0 nan
score nan
(168, 6)
(211, 6)


KeyboardInterrupt: 

In [ ]:
estimators.ipw_fit(folds=5, algorithm='Ridge', self_norm=False)

In [114]:
a, b= np.where(estimators.A == 1)

In [115]:
np.mean(b == np.argmax(estimators.bpol_array, axis=1))

0.8285714285714286

In [116]:
b == np.argmax(estimators.bpol_array, axis=1)

array([ True,  True, False, False, False,  True,  True,  True,  True,
       False,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True, False,
        True,  True,  True,  True,  True, False,  True, False,  True,
        True, False,  True,  True,  True,  True, False,  True,  True,
       False, False,  True,  True,  True,  True, False,  True,  True,
       False,  True,  True, False,  True,  True, False,  True,  True,
        True,  True, False,  True,  True,  True,  True,  True, False,
       False,  True,  True,  True,  True,  True,  True,  True, False,
        True,  True,  True,  True, False,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True, False,  True, False,  True,
        True,  True,  True,  True,  True, False,  True,  True,  True,
        True,  True,  True, False, False, False,  True,  True,  True,
        True,  True,

In [126]:
estimators.fit()

(210, 42)
(210, 42)
(210, 42)
(210, 42)
(210, 42)
5
4
1
(168, 210)
(168, 211)
(168, 6)
(211, 6)
score0 0.5920741597046222
score 0.22304838279334344
(168, 6)
(211, 6)
score0 0.5484855666994175
score 0.21996685751965783
(168, 6)
(211, 6)
score0 0.4370257580785539
score 0.21936581372995897
(168, 6)
(211, 6)
score0 0.38258719055318824
score 0.1960691619150928
(168, 6)
(211, 6)
score0 0.1453535381565161
score 0.0998240228390467
(168, 6)
(211, 6)
score0 0.1176025078946443
score 0.08761088145672416
(168, 6)
(211, 6)
score0 0.11558404389026423
score 0.08667139715272348
(168, 6)
(211, 6)
score0 0.11538739056311095
score 0.08657940390012661
5
4
1
(168, 210)
(168, 211)
(168, 6)
(211, 6)
score0 0.5282926747853551
score 0.34780252844829007
(168, 6)
(211, 6)
score0 0.5109027912907325
score 0.3399568409582776
(168, 6)
(211, 6)
score0 0.4055612035494752
score 0.34152450024979086
(168, 6)
(211, 6)
score0 0.3520824636363296
score 0.29516334917349535
(168, 6)
(211, 6)
score0 0.13652752334001284
score 0.1

score0 0.10340642246564873
score 0.1432496496713782
(168, 6)
(211, 6)
score0 0.10178968775375105
score 0.14173865633599808
(168, 6)
(211, 6)
score0 0.10163179680237974
score 0.14159080397879392
5
4
1
(168, 210)
(168, 211)
(168, 6)
(211, 6)
score0 0.48996644681547585
score 0.45161110390632925
(168, 6)
(211, 6)
score0 0.478927804877456
score 0.444495064050424
(168, 6)
(211, 6)
score0 0.37994781838080577
score 0.4400237766512656
(168, 6)
(211, 6)
score0 0.32715293594272604
score 0.36969767079655697
(168, 6)
(211, 6)
score0 0.13187744350696867
score 0.12821720377353985
(168, 6)
(211, 6)
score0 0.11266672617861144
score 0.10600801396521792
(168, 6)
(211, 6)
score0 0.11114809163040432
score 0.10428776725216349
(168, 6)
(211, 6)
score0 0.11099921483412137
score 0.10411943064699715
(210, 42)
(210, 42)
(210, 42)
(210, 42)
(210, 42)
5
4
1
(168, 210)
(168, 211)
(168, 6)
(211, 6)
score0 0.6604764638334212
score 0.48424140171646374
(168, 6)
(211, 6)
score0 0.6545312410188002
score 0.480274336497344

score0 0.3606097410062051
score 0.5613375391952614
(168, 6)
(211, 6)
score0 0.35659671394840203
score 0.5541195129032266
(168, 6)
(211, 6)
score0 0.3283484932769597
score 0.5033856026279437
(168, 6)
(211, 6)
score0 0.15523084591509753
score 0.19276698860819366
(168, 6)
(211, 6)
score0 0.11164577519658936
score 0.11801345535546867
5
4
1
(168, 210)
(168, 211)
(168, 6)
(211, 6)
score0 0.3972607356195228
score 0.41823785360512017
(168, 6)
(211, 6)
score0 0.3972585610509419
score 0.4182355436084012
(168, 6)
(211, 6)
score0 0.3971889028874397
score 0.4181748775881799
(168, 6)
(211, 6)
score0 0.396648034442802
score 0.41766552262096823
(168, 6)
(211, 6)
score0 0.39253841105612397
score 0.4138270728455681
(168, 6)
(211, 6)
score0 0.363568491197403
score 0.3867385713965819
(168, 6)
(211, 6)
score0 0.17131226422761484
score 0.20694876361720982
(168, 6)
(211, 6)
score0 0.10540771933408649
score 0.14519309125381058
5
4
1
(168, 210)
(168, 211)
(168, 6)
(211, 6)
score0 0.3885570455560412
score 0.453

NameError: name 'linear_model' is not defined

In [ ]:
np.max(pi_behavior_seq_train, axis=0)

In [58]:
from statsmodels.nonparametric.kernel_regression import KernelReg

In [114]:
X, Y, Y_matrix, train_test_split, x_prob, classes, N, N_train, N_test = data_generation(data_name)

In [88]:
from sklearn.kernel_ridge import KernelRidge

In [89]:
clf = KernelRidge(alpha=0.01)
clf.fit(X, Y)

KernelRidge(alpha=0.01, coef0=1, degree=3, gamma=None, kernel='linear',
            kernel_params=None)

In [106]:
from sklearn import linear_model

In [115]:
clf = linear_model.LogisticRegression(penalty='l1', C=0.1, solver='saga', multi_class='multi_class')

In [116]:
Y

array([8, 2, 1, ..., 5, 1, 7])

In [117]:
a, b = np.where(Y_matrix == 1)

In [118]:
b

array([8, 2, 1, ..., 5, 1, 7])

In [120]:
Y

array([8, 2, 1, ..., 5, 1, 7])

In [25]:
model = KernelReg([1,100,1], X, var_type='c')

ValueError: cannot reshape array of size 159660 into shape (4435,1)

In [28]:
import statsmodels.api as sm

In [33]:
'3'*3

'333'

In [35]:
'c'*X.shape[1]

'cccccccccccccccccccccccccccccccccccc'

In [36]:
dens_u = sm.nonparametric.KDEMultivariate(data=X, var_type='c'*dim, bw='normal_reference')

In [37]:
dens_u.pdf(X[0])

array(0.56532044)

In [69]:
bw = dens_u.bw

In [102]:
Y = Y[:50]
X = X[:50]

In [103]:
dens_u = sm.nonparametric.KDEMultivariate(data=X, var_type='c'*X.shape[1], bw='normal_reference')

In [104]:
model = KernelReg(Y, X, var_type='c'*X.shape[1], reg_type='lc')

In [98]:
model.bw

array([0.19174831, 0.28450028, 0.27009914, 0.15901443, 0.30847048,
       0.28977343, 0.17193598, 0.03891559, 0.20149673, 0.31748389,
       0.28018324, 0.28895554, 0.15374873, 0.21203015, 0.57902868,
       0.05410153])

In [85]:
v, x = model.fit(X)

In [87]:
x.shape

(3000, 16)

In [60]:
X.size

119904

In [55]:
model.fit(X[-16:])

(array([4.73930242, 4.83120944, 4.76830902, 4.05162109, 4.54722321,
        4.90765989, 4.45411434, 4.89109568, 4.71458883, 4.78442258,
        4.7987667 , 4.75148552, 4.46570638, 4.97899378, 4.98634629,
        4.81090934]),
 array([[-2.42074379e+15, -2.42074379e+15, -2.42074379e+15,
         -2.42074379e+15, -2.42074379e+15, -2.42074379e+15,
         -2.42074379e+15, -2.42074379e+15, -2.42074379e+15,
         -2.42074379e+15, -2.42074379e+15, -2.42074379e+15,
         -2.42074379e+15, -2.42074379e+15, -2.42074379e+15,
         -2.42074379e+15, -2.42074379e+15, -2.42074379e+15,
         -2.42074379e+15, -2.42074379e+15, -2.42074379e+15,
         -2.42074379e+15, -2.42074379e+15, -2.42074379e+15,
         -2.42074379e+15, -2.42074379e+15, -2.42074379e+15,
         -2.42074379e+15, -2.42074379e+15, -2.42074379e+15,
         -2.42074379e+15, -2.42074379e+15, -2.42074379e+15,
         -2.42074379e+15, -2.42074379e+15, -2.42074379e+15],
        [-1.80097159e+16, -1.80097159e+16, -1.8009715

In [26]:
X.shape

(4435, 36)

In [24]:
model.fit([1])

(array([1.]), array([[-99.]]))

In [14]:
Y

array([0, 0, 0, ..., 5, 5, 5])

In [4]:
num_trials = 100
alphas = [0.7, 0.4, 0.0]

tau_list = np.zeros(num_trials)
res_ipw3_list = np.zeros((num_trials, len(alphas)))
res_dm_list = np.zeros((num_trials, len(alphas)))
res_dml1_truew_list = np.zeros((num_trials, len(alphas)))
res_dml2_truew_list = np.zeros((num_trials, len(alphas)))

for trial in range(num_trials):
    X, Y, Y_matrix, train_test_split, x_prob, classes, N, N_train, N_test = data_generation(data_name)

    X_train, X_test = X[train_test_split], X[~train_test_split]
    Y_train, Y_test = Y[train_test_split], Y[~train_test_split]
    Y_matrix_train, Y_matrix_test = Y_matrix[train_test_split], Y_matrix[~train_test_split]

    pi_behavior, pi_evaluation  = behavior_and_evaluation_policy(X, Y, train_test_split, classes, alpha=0.7)

    pi_behavior_train, pi_behavior_test = pi_behavior[train_test_split], pi_behavior[~train_test_split]
    pi_evaluation_train, pi_evaluation_test = pi_evaluation[train_test_split], pi_evaluation[~train_test_split]

    tau = true_value(Y_matrix_test, pi_evaluation_test, N_test)

    for idx_alpha in  range(len(alphas)):    
        alpha = alphas[idx_alpha]
        pi_behavior, pi_evaluation  = behavior_and_evaluation_policy(X, Y, train_test_split, classes, alpha=alpha)

        perm = np.random.permutation(N_train)

        X_seq_train, Y_matrix_seq_train, pi_behavior_seq_train, pi_evaluation_seq_train = X_train[perm], Y_matrix_train[perm], pi_behavior_train[perm], pi_evaluation_train[perm]

        Y_historical_matrix = np.zeros(shape=(N_train, len(classes)))
        A_historical_matrix = np.zeros(shape=(N_train, len(classes)))

        for i in range(N_train):
            a = np.random.choice(classes, p=pi_behavior[i])
            Y_historical_matrix[i, a] = 1
            A_historical_matrix[i, a] = 1
            
        #IPW3 estimator
        res_ipw3 = ipw(Y_historical_matrix, X_seq_train, X_test, classes, pi_evaluation_train, A_hst=A_historical_matrix)
        #Direct method estimator
        res_dm = dm(Y_historical_matrix, X_seq_train, X_test, pi_evaluation_test, classes)
        #DML with L1
        res_dml1 =dml(Y_historical_matrix, A_historical_matrix, X_seq_train, X_test, pi_evaluation_seq_train, pi_evaluation_test, classes, pi_behavior=pi_behavior_seq_train, method='Lasso')
        #DML with L2
        res_dml2 =dml(Y_historical_matrix, A_historical_matrix, X_seq_train, X_test, pi_evaluation_seq_train, pi_evaluation_test, classes, pi_behavior=pi_behavior_seq_train, method='Ridge')

        print(res_ipw3)
        print(res_dm)
        print(res_dr1)
        print(res_dr2)
        print(res_dml1_truew)
        print(res_dml2_truew)
        
        res_ipw3_list[trial, idx_alpha] = res_ipw3
        res_dm_list[trial, idx_alpha] = res_dm
        res_dr1_list[trial, idx_alpha] = res_dr1
        res_dr2_list[trial, idx_alpha] = res_dr2
        res_dml1_truew_list[trial, idx_alpha] = res_dml1_truew
        res_dml2_truew_list[trial, idx_alpha] = res_dml2_truew
        
        np.savetxt("exp_results/res_ipw3.csv", res_ipw3_list, delimiter=",")
        np.savetxt("exp_results/res_dm.csv", res_dm_list, delimiter=",")
        np.savetxt("exp_results/res_dr1.csv", res_dr1_list, delimiter=",")
        np.savetxt("exp_results/res_dr2.csv", res_dr2_list, delimiter=",")
        np.savetxt("exp_results/res_dml1.csv", res_dml1_truew_list, delimiter=",")
        np.savetxt("exp_results/res_dml2.csv", res_dml2_truew_list, delimiter=",")
        
    tau_list[trial] = tau
        

[[0.2857443  0.1952922  0.26814298 0.07805563 0.05606263 0.11670226]
 [0.29536795 0.15428734 0.27987302 0.0802635  0.05640643 0.13380176]
 [0.37533892 0.1459109  0.18370155 0.12563121 0.06332002 0.10609741]
 ...
 [0.20679231 0.16011928 0.29419599 0.0838981  0.1119685  0.14302582]
 [0.31595816 0.13683452 0.25687274 0.10179922 0.11506038 0.07347497]
 [0.28837575 0.1668954  0.27622361 0.06731022 0.08418668 0.11700835]]
sigma:  0.001
lda:  0.01
sigma:  0.001
lda:  0.01
sigma:  0.001
lda:  0.01
sigma:  0.001
lda:  0.01
sigma:  0.001
lda:  0.01


/Users/mkato/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.08136092229213432, tolerance: 0.04414618372652263
  positive)
/Users/mkato/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.08136092229213432, tolerance: 0.04414618372652263
  positive)
/Users/mkato/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.09048368117350947, tolerance: 0.04430395631018423
  positive)
/Users/mkato/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You mi

sigma:  0.001
lda:  0.01
sigma:  0.001
lda:  0.01
sigma:  0.001
lda:  0.01
sigma:  0.001
lda:  0.01
sigma:  0.001
lda:  0.01
sigma:  0.001
lda:  0.01
sigma:  0.001
lda:  0.01
RuLSIF starting...
Searching for the optimal sigma and lambda...
sigma = 0.00100, lambda = 0.00100, score = 0.00000
sigma = 0.00100, lambda = 0.01000, score = 0.00000
sigma = 0.00100, lambda = 0.10000, score = 0.00000
sigma = 0.00100, lambda = 1.00000, score = 0.00000
sigma = 0.00100, lambda = 10.00000, score = -0.00000
sigma = 0.00100, lambda = 100.00000, score = 0.00000
sigma = 0.00100, lambda = 1000.00000, score = 0.00000
sigma = 0.00100, lambda = 10000.00000, score = -0.00000
sigma = 0.00100, lambda = 100000.00000, score = -0.00000
sigma = 0.00100, lambda = 1000000.00000, score = 0.00000
sigma = 0.00100, lambda = 10000000.00000, score = 0.00000
sigma = 0.00100, lambda = 100000000.00000, score = -0.00000
sigma = 0.00100, lambda = 1000000000.00000, score = 0.00000
sigma = 0.01000, lambda = 0.00100, score = -0.00

sigma = 100000000.00000, lambda = 0.00100, score = -0.50000
sigma = 100000000.00000, lambda = 0.01000, score = -0.50000
sigma = 100000000.00000, lambda = 0.10000, score = -0.50000
sigma = 100000000.00000, lambda = 1.00000, score = -0.49994
sigma = 100000000.00000, lambda = 10.00000, score = -0.49581
sigma = 100000000.00000, lambda = 100.00000, score = -0.37491
sigma = 100000000.00000, lambda = 1000.00000, score = -0.08677
sigma = 100000000.00000, lambda = 10000.00000, score = -0.00985
sigma = 100000000.00000, lambda = 100000.00000, score = -0.00100
sigma = 100000000.00000, lambda = 1000000.00000, score = -0.00010
sigma = 100000000.00000, lambda = 10000000.00000, score = -0.00001
sigma = 100000000.00000, lambda = 100000000.00000, score = -0.00000
sigma = 100000000.00000, lambda = 1000000000.00000, score = -0.00000
sigma = 1000000000.00000, lambda = 0.00100, score = -0.50000
sigma = 1000000000.00000, lambda = 0.01000, score = -0.50000
sigma = 1000000000.00000, lambda = 0.10000, score = -

sigma = 1000000.00000, lambda = 0.00100, score = -0.50000
sigma = 1000000.00000, lambda = 0.01000, score = -0.50000
sigma = 1000000.00000, lambda = 0.10000, score = -0.50000
sigma = 1000000.00000, lambda = 1.00000, score = -0.49994
sigma = 1000000.00000, lambda = 10.00000, score = -0.49581
sigma = 1000000.00000, lambda = 100.00000, score = -0.37491
sigma = 1000000.00000, lambda = 1000.00000, score = -0.08677
sigma = 1000000.00000, lambda = 10000.00000, score = -0.00985
sigma = 1000000.00000, lambda = 100000.00000, score = -0.00100
sigma = 1000000.00000, lambda = 1000000.00000, score = -0.00010
sigma = 1000000.00000, lambda = 10000000.00000, score = -0.00001
sigma = 1000000.00000, lambda = 100000000.00000, score = -0.00000
sigma = 1000000.00000, lambda = 1000000000.00000, score = -0.00000
sigma = 10000000.00000, lambda = 0.00100, score = -0.50000
sigma = 10000000.00000, lambda = 0.01000, score = -0.50000
sigma = 10000000.00000, lambda = 0.10000, score = -0.50000
sigma = 10000000.00000, 

...

sigma:  0.001
lda:  0.01
sigma:  0.001
lda:  0.01
sigma:  0.001
lda:  0.01
sigma:  0.001
lda:  0.01
sigma:  0.001
lda:  0.01
sigma:  0.001
lda:  0.01
sigma:  0.001
lda:  0.01
sigma:  0.001
lda:  0.01
sigma:  0.001
lda:  0.01
sigma:  0.001
lda:  0.01
sigma:  0.001
lda:  0.01
RuLSIF starting...
Searching for the optimal sigma and lambda...
sigma = 0.00100, lambda = 0.00100, score = 0.00000
sigma = 0.00100, lambda = 0.01000, score = 0.00000
sigma = 0.00100, lambda = 0.10000, score = 0.00000
sigma = 0.00100, lambda = 1.00000, score = 0.00000
sigma = 0.00100, lambda = 10.00000, score = -0.00000
sigma = 0.00100, lambda = 100.00000, score = 0.00000
sigma = 0.00100, lambda = 1000.00000, score = 0.00000
sigma = 0.00100, lambda = 10000.00000, score = -0.00000
sigma = 0.00100, lambda = 100000.00000, score = -0.00000
sigma = 0.00100, lambda = 1000000.00000, score = 0.00000
sigma = 0.00100, lambda = 10000000.00000, score = 0.00000
sigma = 0.00100, lambda = 100000000.00000, score = -0.00000
sigma = 

sigma = 100000000.00000, lambda = 0.00100, score = -0.50000
sigma = 100000000.00000, lambda = 0.01000, score = -0.50000
sigma = 100000000.00000, lambda = 0.10000, score = -0.50000
sigma = 100000000.00000, lambda = 1.00000, score = -0.49994
sigma = 100000000.00000, lambda = 10.00000, score = -0.49581
sigma = 100000000.00000, lambda = 100.00000, score = -0.37491
sigma = 100000000.00000, lambda = 1000.00000, score = -0.08677
sigma = 100000000.00000, lambda = 10000.00000, score = -0.00985
sigma = 100000000.00000, lambda = 100000.00000, score = -0.00100
sigma = 100000000.00000, lambda = 1000000.00000, score = -0.00010
sigma = 100000000.00000, lambda = 10000000.00000, score = -0.00001
sigma = 100000000.00000, lambda = 100000000.00000, score = -0.00000
sigma = 100000000.00000, lambda = 1000000000.00000, score = -0.00000
sigma = 1000000000.00000, lambda = 0.00100, score = -0.50000
sigma = 1000000000.00000, lambda = 0.01000, score = -0.50000
sigma = 1000000000.00000, lambda = 0.10000, score = -

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/mkato/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-4-afe327fa7d7e>", line 45, in <module>
    res_dr2 = dr(Y_historical_matrix, A_historical_matrix, X_seq_train, X_test, pi_evaluation_seq_train, pi_evaluation_test, classes, pi_behavior=pi_behavior_seq_train, method='Ridge')
  File "../cs_ope_estimator.py", line 79, in dr
    densratio_obj = densratio(X_evl, X_hst)
  File "/Users/mkato/opt/anaconda3/lib/python3.7/site-packages/densratio/core.py", line 66, in densratio
    result = RuLSIF(x, y, alpha, sigma_range, lambda_range, kernel_num, verbose)
  File "/Users/mkato/opt/anaconda3/lib/python3.7/site-packages/densratio/RuLSIF.py", line 61, in RuLSIF
    opt_params = search_sigma_and_lambda(x, y, alpha, centers, sigma_range, lambda_range, verbose)
  File "/Users/mkato/opt/anaconda3/lib/python3.7/site-packages

KeyboardInterrupt: 